In [8]:
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import *
from sklearn.preprocessing import MinMaxScaler

In [2]:
data = pd.read_csv("sales_data_training.csv",dtype = float)

In [3]:
data.head()

,critic_rating,is_action,is_exclusive_to_us,is_portable,is_role_playing,is_sequel,is_sports,suitable_for_kids,total_earnings,unit_price
0,3.5,1.0,0.0,1.0,0.0,1.0,0.0,0.0,132717.0,59.99
1,4.5,0.0,0.0,0.0,0.0,1.0,1.0,0.0,83407.0,49.99
2,3.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,62423.0,49.99
3,4.5,1.0,0.0,0.0,0.0,0.0,0.0,1.0,69889.0,39.99
4,4.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,161382.0,59.99


In [4]:
X = data.drop('total_earnings',axis = 1).values
X[:2]

array([[  3.5 ,   1.  ,   0.  ,   1.  ,   0.  ,   1.  ,   0.  ,   0.  ,
         59.99],
       [  4.5 ,   0.  ,   0.  ,   0.  ,   0.  ,   1.  ,   1.  ,   0.  ,
         49.99]])

In [5]:
Y = data[['total_earnings']].values
Y[:5]

array([[ 132717.],
       [  83407.],
       [  62423.],
       [  69889.],
       [ 161382.]])

In [9]:
test =pd.read_csv("sales_data_test.csv",dtype = float)
X_test = test.drop('total_earnings',axis = 1).values


Y_test = test[['total_earnings']].values

In [10]:
X_scaler = MinMaxScaler(feature_range= (0,1))
Y_scaler = MinMaxScaler(feature_range= (0,1))

In [11]:
X_train = X_scaler.fit_transform(X)
Y_train = Y_scaler.fit_transform(Y)

X_test = X_scaler.transform(X_test)
Y_test = Y_scaler.transform(Y_test)

In [12]:
print("Note: Y values were scaled by multiplying by {:.10f} and adding {:.4f}".format(Y_scaler.scale_[0], Y_scaler.min_[0]))

Note: Y values were scaled by multiplying by 0.0000036968 and adding -0.1159


### Build a neural network with Keras

In [13]:
# Define the model with an input layer, 3 hidden layers and an output layer
model = Sequential()
model.add(Dense(50, input_dim=9, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(1, activation='linear'))
model.compile(loss='mean_squared_error', optimizer='adam')

In [16]:
# TensorBoard logging
logger = keras.callbacks.TensorBoard(
    log_dir = "logs/keras/",
    write_graph = True
)

In [17]:
# Train the model
model.fit(
    X_train,
    Y_train,
    epochs=50,
    shuffle=True,
    verbose=0,
    callbacks = [logger]
)

In [18]:
test_error_rate = model.evaluate(X_test, Y_test, verbose=0)
print("MSE: {}".format(test_error_rate))

MSE: 0.00013755901280092077


## Predict

In [19]:
X = pd.read_csv("proposed_new_product.csv").values

In [20]:
prediction = model.predict(X)

In [21]:
prediction = prediction[0][0]

In [22]:
prediction = prediction + 0.1159
prediction = prediction / 0.0000036968

In [23]:
print("Earnings Prediction for Proposed Product is ${}".format(prediction))

Earnings Prediction for Proposed Product is $261288.9324837086


## Save

In [24]:
# Save model in h d 5 format
model.save("trained_model.h5")

In [ ]:
# can be loaded with
# model = load_model("trained_model.h5")